In [8]:
from __future__ import division
import requests
import csv
import yaml
import time
import pandas as pd
import numpy as np
import time
from easymail import utils_sendMail

In [2]:
# load the server credentials
configs = yaml.load(open('server_config.yaml'))


In [ ]:
# main
while True:
    try:
        for credential in configs.values():
            # query dstat server
            res = query_heartbeat_server(credential)
            hostname = res['hostname']

            # check status
            det = check_server_down(res)
            
            if det:
                if check_alert_met(hostname):
                    send_alert(hostname, 'email_list.csv')

                        
    # This happen when the server down, so continue to avoid the client_server crash.
    except KeyError:
        continue

In [22]:
for credential in configs.values():
    # query dstat server
    res = query_heartbeat_server(credential)
    hostname = res['hostname']

    # check status
    det = check_server_down(res)

    if det:
        if check_alert_met(hostname):
            send_alert(hostname, 'email_list.csv')

### Design Functions

In [3]:
# query heartbeat server
def query_heartbeat_server(pack):
    output = {'hostname': pack['hostname']}
    url = url = pack['url'].strip('/') +'/heartbeat'
    try:
        r = requests.post(url, json=pack, timeout=0.5)
        if r.status_code != 200:
            output['content'] = r.content
            output['status'] = 'failed'
            return output
        ans = r.json()
        output.update(ans)
        if(output["verification"] != "successful"):
            output['content'] = r.content
            output['status'] = 'failed'
            flash("verification failed!")
            return output
        output['status'] = 'success'

        return output   
    except Exception, e:
        output['content'] = repr(e)
        output['status'] = 'failed'
        return output
    # end try
# end def 

In [4]:
def check_server_down(res):
    status = res['status']
    if status != 'success':
        return True

In [19]:
# to check if necessary to send email
def check_alert_met(hostname, frequency=2):
    
    
    def write_log(hostname, send_email='no'):
        try:
            df = pd.read_csv(hostname+ '_server_down.csv', index_col=0)
            log = {"time": time.time(), 'email': send_email}
            df = df.append(log, ignore_index=True)
            df.to_csv(hostname+ '_server_down.csv')

        # when csv not exist
        except IOError:
            log = {"time": time.time(), 'email': send_email}
            _df = pd.DataFrame(log, index=[0])
            _df.to_csv(hostname+ '_server_down.csv')
    
    
    try: 
        df = pd.read_csv(hostname+ '_server_down.csv', index_col=0).sort_values('time')
        
            
        # only trace the data based on frequency
        trace_back_time = frequency * 60 # in seconds
        now = time.time()
        cutoff = now - trace_back_time + 1
        _df = df[df['time'] >= cutoff]
        if 'yes' not in _df['email'].values:
            # write log
            write_log(hostname, send_email='yes')
            # return True to send email
            return True
            
        else:
            # write log but not sending email
            write_log(hostname, send_email='no')
            return False
            
    # when file not exist        
    except IOError:
        write_log(hostname, send_email='no')
        return False

In [11]:
def send_alert(hostname, recipient_email_filename):

    # make subject of email
    subject = "Server " + hostname + " is down."
        
    # make text    
    text = "Server " + hostname + " is down at " + str(time.time())
    
    
    with open(recipient_email_filename) as f:
        emailReader = csv.reader(f)
        emailList = list(emailReader)
    for email in emailList:
        print text
        #utils_sendMail('dstat_client', email[0], subject, text, attachFile = None)